In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re
from collections import Counter

URLs Go Here

In [2]:
job_urls = [
    "https://jobs.lever.co/palantir/76a60923-bb49-40f5-b061-7c7eb1299602",
    "https://jobs.lever.co/palantir/bc5c4098-07d5-49ed-a398-a27e6191aa30",
    "https://jobs.lever.co/voleon/bdc0c822-f9fa-47d3-bba1-3a25fcc6b6e2",
    "https://jobs.lever.co/theodo/1592b48f-9207-46f2-908e-d835cc6121ad",
    "https://job-boards.greenhouse.io/mthreerecruitingportal/jobs/4401897006",
    "https://job-boards.eu.greenhouse.io/quantboxresearchpte/jobs/4228244101",
    "https://jobs.lever.co/apolloresearch/64a79893-84bf-4005-a762-852ee2bcccce",
    "https://job-boards.eu.greenhouse.io/aisi/jobs/4386112101",
    "https://job-boards.eu.greenhouse.io/copperco/jobs/4639292101",
    "https://jobs.lever.co/palantir/d372c805-d0cd-4a10-9522-fbecc78d6f3e",
    "https://job-boards.greenhouse.io/nx5j6ba7z1p2ujjnx5j6ba7z1p2ujj/jobs/5536985004",
    "https://job-boards.greenhouse.io/figma/jobs/5551697004?gh_jid=5551697004",
    "https://job-boards.greenhouse.io/speechify/jobs/5612196004",
    "https://job-boards.greenhouse.io/speechify/jobs/5614710004",
    "https://job-boards.greenhouse.io/clearstreet/jobs/6884339",
    "https://jobs.lever.co/olo/202c058d-f2bc-4cd9-9d25-6ff3164f5812",
    "https://job-boards.greenhouse.io/via/jobs/7758764002",
    "https://job-boards.eu.greenhouse.io/physicsx/jobs/4652615101",
    "https://job-boards.greenhouse.io/speechify/jobs/5614725004",
    "https://jobs.lever.co/palantir/c44510a1-9537-4c52-ae81-51546979fe47",
    "https://job-boards.greenhouse.io/speechify/jobs/5614010004",
    "https://job-boards.greenhouse.io/speechify/jobs/5614747004",
    "https://job-boards.greenhouse.io/speechify/jobs/5613965004",
    "https://job-boards.greenhouse.io/mattermost/jobs/4885981008?gh_src=kKpezZ",
    "https://jobs.lever.co/palantir/5168e8fd-fec1-4fea-b7a1-81bdaea65850",
    "https://job-boards.eu.greenhouse.io/polyai/jobs/4653835101",
    "https://job-boards.eu.greenhouse.io/mast/jobs/4060297101",
    "https://job-boards.eu.greenhouse.io/quantboxresearchpte/jobs/4231150101",
    "https://job-boards.greenhouse.io/pharomanagement/jobs/6489121003",
    "https://job-boards.eu.greenhouse.io/nothing/jobs/4639317101",
    "https://jobs.lever.co/crypto/bedb489a-a587-4a3b-ae4e-76daa1d7b98a",
    "https://job-boards.greenhouse.io/fiveringsllc/jobs/4817027008",
    "https://job-boards.greenhouse.io/infomedia/jobs/6607921003",
    "https://jobs.lever.co/octoenergy/b841ead2-9d99-487a-a150-bf355a1e4fdb",
    "https://job-boards.greenhouse.io/speechify/jobs/5614742004",
    "https://job-boards.greenhouse.io/speechify/jobs/5614715004",
    "https://job-boards.eu.greenhouse.io/modoenergy/jobs/4516559101",
    "https://job-boards.eu.greenhouse.io/physicsx/jobs/4644847101",
    "https://jobs.lever.co/palantir/3f4dd595-2f57-4b5f-a50a-8d3084641ef8",
    "https://job-boards.greenhouse.io/janestreet/jobs/6236002002",
    "https://job-boards.greenhouse.io/dataikujobs/jobs/5420320004",
    "https://job-boards.greenhouse.io/speechify/jobs/5614736004",
    "https://job-boards.eu.greenhouse.io/physicsx/jobs/4652622101",
    "https://job-boards.greenhouse.io/sevenroomsuk/jobs/7117699",
    "https://job-boards.greenhouse.io/xtxmarketstechnologies/jobs/6593467003"
    "https://jobs.lever.co/Yassir/13c90265-5c71-492d-a570-9c3af60282b0",
    "https://job-boards.greenhouse.io/clickhouse/jobs/5492195004",
    "https://job-boards.greenhouse.io/andurilindustries/jobs/4544851007?gh_jid=4544851007",
    "https://job-boards.greenhouse.io/xtxmarketstechnologies/jobs/5901218003",
    "https://job-boards.greenhouse.io/addepar1/jobs/8076643002",
    "https://job-boards.greenhouse.io/blackduck/jobs/4822383008?gh_src=F-Prime+Capital+Partners+job+board"
    
]

Terms and Lists 

In [3]:
level_terms = ["intern", "graduate", "entry", "entry-level", "early career", "junior", "new grad"]
tech_terms = ["software", "developer", "engineer", "fullstack", "frontend", "backend", "devops", "platform"]
skills_keywords = ['python', 'java', 'c++', 'c#', '.net', 'ruby', 'Go', 'r', 'scala', 'Swift', 'kotlin', 'javascript', 'typescript', 'node', 'react', 'angular', 'vue.js', 'django', 'flask', 'spring', 'express.js', 'html', 'css', 'sql', 'postgresql', 'mysql', 'mongodb', 'cassandra', 'redis', 'spark', 'hadoop', 'aws', 'azure', 'gcp', 'docker', 'kubernetes', 'terraform', 'ansible', 'jenkins', 'gitlab ci/cd', 'github actions', 'aws lambda', 'azure functions', 'git', 'svn', 'rabbitmq', 'kafka', 'linux', 'llm', 'data structures', 'algorithms']

In [4]:
def is_target_role(title, description):
    title = title.lower()
    description = description.lower()
    return (
        any(kw in title for kw in level_terms) and
        any(t in title or t in description for t in tech_terms)
    )

def extract_skills(text):
    text = text.lower()
    return [kw for kw in skills_keywords if kw in text]

Main Function

In [ ]:
def parse_job_url(url):
    path = urlparse(url).path.strip("/")
    domain = urlparse(url).netloc

    if "greenhouse.io" in domain:
        match = re.match(r"([^/]+)/jobs/(\d+)", path)
        if match:
            return {"platform": "greenhouse", "company": match.group(1), "job_id": match.group(2)}

    elif "lever.co" in domain:
        match = re.match(r"([^/]+)/([a-zA-Z0-9]+)", path)
        if match:
            return {"platform": "lever", "company": match.group(1), "job_token": match.group(2)}

    return None

def dispatch_scraper(url):
    parsed = parse_job_url(url)
    if not parsed:
        print(f"Unsupported or unparseable URL: {url}")
        return None

    if parsed["platform"] == "greenhouse":
        return scrape_greenhouse(parsed["company"], parsed["job_id"], url)

    elif parsed["platform"] == "lever":
        return scrape_lever(parsed["company"], parsed["job_token"], url)

    print(f"Unknown platform for: {url}")
    return None

Greenhouse

In [ ]:
def scrape_greenhouse(company, job_id, url):
    api_url = f"https://boards-api.greenhouse.io/v1/boards/{company}/jobs/{job_id}"
    resp = requests.get(api_url)
    if resp.status_code != 200:
        print(f"Failed to fetch Greenhouse job: {url}")
        return None

    data = resp.json()
    title = data.get("title", "")
    location = data.get("location", {}).get("name", "Unknown").lower()
    description_html = data.get("content", "")
    description = BeautifulSoup(description_html, "html.parser").get_text()

    #if not is_target_role(title, description):
        #print(f"Skipped non-target role: {title} ({location})")
        #return None

    return {
        "title": title,
        "company": company,
        "location": location,
        "url": url,
        "description": description,
        "skills": extract_skills(description)
    }


Lever

In [ ]:
def scrape_lever(company, job_token, url):
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Failed to fetch Lever job: {url}")
        return None

    soup = BeautifulSoup(resp.text, "html.parser")

    # Try multiple strategies for extracting the title
    title_tag = (
    soup.find("h2", class_="posting-headline") or
    soup.find("h1") or
    soup.find("h2") or
    soup.title
)
    title = title_tag.get_text(strip=True) if title_tag else "Unknown Title"


    location_tag = soup.find("div", class_="location")
    location = location_tag.get_text(strip=True) if location_tag else "Unknown"

    desc_tag = soup.find("div", class_="section page-centered")
    description = desc_tag.get_text(separator=" ", strip=True) if desc_tag else ""

    #if not is_target_role(title, description):
        #print(f"Skipped non-target role: {title} ({location})")
        #return None

    return {
        "title": title,
        "company": company,
        "location": location.lower(),
        "url": url,
        "description": description,
        "skills": extract_skills(description)
    }
    
jobs_data = []
for url in job_urls:
    result = dispatch_scraper(url)
    if result:
        jobs_data.append(result)

In [ ]:
df = pd.DataFrame(jobs_data)
df.to_csv("Greenhouse&Lever_Scraped_Jobs.csv", index=False)
print("Job data saved to Greenhousee&Lever_Scraped_Jobs.csv")
print(f"Total jobs scraped: {len(df)}")

Keyword Frequency Analysis

In [ ]:
all_skills = [s for sublist in df['skills'] for s in sublist]
skill_freq = Counter(all_skills)
freq_df = pd.DataFrame(list(skill_freq.items()), columns=['Skill', 'Frequency']).sort_values(by='Frequency', ascending=False)
freq_df.to_csv("Greenhouse&Lever_Skill_Frequency.csv", index=False)
print("Skill frequency saved to Greenhouse&Lever_Skill_Frequency.csv")